# Introducción

El sector bancario enfrenta desafíos significativos relacionados con la retención de clientes, un problema que afecta directamente la rentabilidad y sostenibilidad de las instituciones financieras. En Beta Bank, la pérdida gradual de clientes representa una preocupación creciente, ya que estudios han demostrado que retener a los clientes existentes es más eficiente y económico que adquirir nuevos. Ante esta situación, el banco busca implementar una solución basada en datos para identificar y predecir a los clientes propensos a abandonar, con el objetivo de tomar medidas preventivas y mejorar la experiencia del cliente.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

# Exploración de dataset

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None


In [4]:
print(data.head())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

En la exploración inicial del dataset se observa que contiene 10,000 registros con 14 columnas, donde la mayoría de las características están completas, excepto la columna Tenure, que presenta algunos valores faltantes. Las variables incluyen información demográfica y financiera de los clientes, como CreditScore, Geography, Gender, Age, Balance, y EstimatedSalary, junto con la variable objetivo Exited, que indica si el cliente dejó el banco. Además, las columnas categóricas como Geography y Gender deberán ser codificadas para el modelo, y se debe abordar el manejo de los valores nulos en Tenure antes de proceder con el análisis y la construcción del modelo de machine learning.

# Preparación de datos

In [5]:
data = pd.get_dummies(data, columns=['Geography', 'Gender'], drop_first=True)

scaler = StandardScaler()
numeric_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])


In [6]:
features = data.drop(columns=['Exited', 'RowNumber', 'CustomerId', 'Surname'])
target = data['Exited']
features.fillna(features.mean(), inplace=True)

Este paso separa las características que el modelo utilizará para hacer predicciones (features) y la variable objetivo que queremos predecir (target). La razón por la que elimine varias columnas, y no solo la columna Exited, es porque algunas de esas columnas no contienen información relevante para la predicción. Además se encontro que features tienen valores nulos, como Random Forest, no pueden trabajar con estos valores. Para solucionarlo, rellenamos los valores faltantes con la media de cada columna o eliminamos las filas incompletas. 

# Dividir el conjunto de datos

In [7]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)

# Examinen de equilibrio de clases

In [8]:

print(target.value_counts(normalize=True))


0    0.7963
1    0.2037
Name: Exited, dtype: float64


Al examinar el equilibrio de clases en el conjunto de datos, observamos un desequilibrio significativo. Aproximadamente el 79.63% de los clientes permanecen con el banco (clase 0), mientras que solo el 20.37% de los clientes han dejado el banco (clase 1). Este desequilibrio sugiere que el modelo podría verse sesgado hacia la clase mayoritaria, prediciendo con mayor frecuencia que los clientes no abandonarán el banco. Para abordar este problema y mejorar la capacidad del modelo para predecir correctamente los clientes que abandonan, será importante implementar técnicas para lidiar con el desequilibrio, como el ajuste de pesos de clase, sobremuestreo de la clase minoritaria o submuestreo de la clase mayoritaria.

# Entrenar el modelo sin abordar el desequilibrio

In [9]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
target_pred = model.predict(features_test)
f1 = f1_score(target_test, target_pred)

print(f'F1-Score sin abordar el desequilibrio: {f1}')

F1-Score sin abordar el desequilibrio: 0.562200956937799



El resultado obtenido de un F1-Score de 0.5622 indica que el modelo de Random Forest sin abordar el desequilibrio de clases no está logrando un buen balance entre precisión y recall. Aunque el rendimiento no está muy lejos del umbral requerido de 0.59, sigue mostrando dificultades para identificar correctamente los clientes que abandonan el banco. Este resultado sugiere que el desequilibrio de clases está afectando la capacidad del modelo para predecir de manera adecuada los casos positivos (clientes que se van), lo que hace necesario aplicar técnicas que ayuden a manejar este desequilibrio, como ajustar los pesos de las clases o utilizar métodos de sobremuestreo.

# Evaluar con AUC-ROC

In [10]:
probabilities = model.predict_proba(features_test)[:, 1]
auc_roc = roc_auc_score(target_test, probabilities)
print(f'AUC-ROC: {auc_roc}')

AUC-ROC: 0.8523906684739958


El AUC-ROC obtenido de 0.852 es un resultado muy positivo. Indica que el modelo tiene una capacidad fuerte para distinguir entre los clientes que abandonan el banco y los que no. Un valor de AUC-ROC cercano a 1 sugiere que el modelo está tomando decisiones precisas en la mayoría de los casos, lo que es un buen indicador del rendimiento general del modelo, especialmente en un escenario con desequilibrio de clases. Este valor es significativamente mejor que un modelo aleatorio, que tendría un AUC-ROC de 0.5.

# Enfoque 1: Ajuste de peso en clase

In [11]:
model_weighted = RandomForestClassifier(random_state=12345, class_weight='balanced')
model_weighted.fit(features_train, target_train)
target_pred_weighted = model_weighted.predict(features_test)

In [12]:
f1_wighted = f1_score(target_test, target_pred_weighted)
probabilities_weighted = model_weighted.predict_proba(features_test)[:,1]
auc_roc_weighted = roc_auc_score(target_test, probabilities_weighted)

print(f'F1-Score con class_weight="balance": {f1_wighted}')
print(f'AUC-ROC con class_weight="balanced": {auc_roc_weighted}')

F1-Score con class_weight="balance": 0.5721096543504172
AUC-ROC con class_weight="balanced": 0.8558036669758151


Los resultados del ajuste de peso de clase utilizando class_weight="balanced" han mejorado ligeramente el rendimiento del modelo. El F1-Score ha subido a 0.5721, lo que representa una mejora respecto al valor inicial, pero aún no alcanza el umbral requerido de 0.59. Sin embargo, el valor de AUC-ROC ha aumentado a 0.8558, lo cual es un buen indicio de que el modelo es capaz de distinguir entre las clases de manera más efectiva. 

# Enfoque 2: Sobremuestreo de la clase minoritaria

In [13]:
def upsample (features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1 ]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled
    
features_upsampled, target_upsampled = upsample(features_train, target_train, 10)


model = RandomForestClassifier(random_state=12345)
model.fit(features_upsampled, target_upsampled)

target_pred =model.predict(features_test)
f1 = f1_score(target_test, target_pred)

probabilities = model.predict_proba(features_test)[:,1]
auc_roc = roc_auc_score(target_test, probabilities)


print(f'F1-Score con sobremuestreo: {f1}')
print(f'AUC-ROC con sobremuestreo: {auc_roc}')

F1-Score con sobremuestreo: 0.6035751840168245
AUC-ROC con sobremuestreo: 0.8496963211338613


En el segundo enfoque de sobremuestreo, logramos mejorar el F1-Score a 0.6035, superando el valor mínimo requerido de 0.59, lo que indica que el modelo está mejor equilibrado para identificar correctamente las clases minoritarias. Además, el AUC-ROC se mantiene en un valor sólido de 0.8496, lo cual refleja que el modelo sigue siendo bastante efectivo en términos de separación entre las clases positivas y negativas. Este enfoque demuestra que el sobremuestreo ayuda a mejorar la capacidad del modelo para predecir correctamente los casos en los que los clientes abandonan el banco, sin sacrificar demasiado la capacidad general de clasificación.

# Resultado final

Enfoque 1: Ajuste de peso en clase
F1-Score: 0.5721
AUC-ROC: 0.8558
Enfoque 2: Sobremuestreo de la clase minoritaria
F1-Score: 0.6036
AUC-ROC: 0.8497

Aunque el AUC-ROC es ligeramente superior en el enfoque de ajuste de pesos (Enfoque 1), el F1-Score es más alto en el enfoque de sobremuestreo (Enfoque 2). Esto indica que el modelo con sobremuestreo tiene un mejor equilibrio entre precisión y recall, lo cual es clave en un escenario con desequilibrio de clases. Por lo tanto, a pesar de la pequeña diferencia en el AUC-ROC, parece que el sobremuestreo de la clase minoritaria es la mejor estrategia en este caso, ya que mejora el F1-Score, lo que indica un mejor rendimiento general en la clasificación de los clientes que abandonan el banco.